# Setup of Noteboook

The follwing code clones the github repository with course files. 
Subsequently it imports all libraries and custom modules needed for this notebook

In [ ]:
!git clone https://github.com/japolak/datahow-course-scripts.git
!pip install --upgrade scipy==1.7.3
!cd /content/datahow-course-scripts

In [101]:
# import libaries
import pandas as pd
import numpy as np
import scipy
import importlib  
import scipy.integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

# import custom modules
emulator = importlib.import_module("datahow-course-scripts.scripts.modules.emulator")
plothelpers = importlib.import_module("datahow-course-scripts.scripts.modules.plothelpers")

# notebook settings
pd.set_option('display.max_rows', None)

# Introduction to Cell Culture Fed-Batch Process Emulator

The emulator is aimed at providing in-silico data to test some of the machine learning tool discussed during the course. The emulator is mimicking the behavior of a fed-batch cell culture process, where only few components are present:


*   The cells, responsible for the production of the product, which are consuming glucose to sustain metabolism and producing lactate as by-product. These are indicated by VCD (viable cell density), typically expressed in million cells / ml.
*   Glucose (Glc) is consumed by the cells and it is continuously fed to the process (F_glc). Too low glucose concentrations are slowing down cell growth and product production. To high glucose concentrations are poisoning the system and accelerating cell death.
*   Lactate (Lac) is a by-product of the cells and it is poisoning the cells, so that too high lactate concentrations are slowing down cell metabolism and accelerating cell death
*   Product (Titer) is produced by the cells. The faster the cells are growing, the less are producing the product. 
*   Aggregates (Aggr) are a by-product of the process and their production is proportianal to the product concentration.

The emulator's results are based on the following equations to create the in-sillico data

### Model Equation Parameters

- Balance on VCD:  $\frac{dVCD}{dt}$ = (μ<sub>g</sub> - μ<sub>d</sub>)VCD
- Balance on Glucose: $\frac{dGlc}{dt}$ = $-k_{Glc} \frac{Glc}{Glc + 0.05}$ VCD + F<sub>Glc</sub>
- Balance on Lactate: $\frac{dLac}{dt}$ = k<sub>Lac</sub> VCD
- Balance on Product: $\frac{dProd}{dt}$ = k<sub>Prod</sub>$\frac{Glc}{Glc + K_{g, Glc} }$ ($\frac{μ_{g}}{μ_{g,max}}$)<sup>2</sup> VCD $-2 \frac{dAggr}{dt}$ 
- Balance on Aggregates: $\frac{dAggr}{dr}$ = k<sub>aggr</sub>Prod<sup>2</sup>

Where:
- Growth rate: $μ_{g} =  μ_{g,max}\frac{Glc}{Glc +K_{g, Glc}}\frac{K_{i, Lac}}{Lac+K_{i, Lac}} $
- Death rate: $μ_{d} = μ_{d,max}(1+\frac{φ}{1+φ})\frac{Lac}{Lac+K_{d, Lac}}$
- Glc saturation: φ = e<sup>0.1(Glc-75)</sup>

The user can change the different rates on the emulator, in order to change the process behavior (please use default values at the beginning)


### Process Parameters

Please insert the values of the process manipulated variables:

- Feed start (day): day at which Glc feed is started
- Feed end (day): ay at which Glc feed is stopped
- Feed rate: mass rate (g/L/day) at which Glc is feed (continuous feed over 24hours)
- Initial Glc concentration (g/L): Glc at time t = 0
- Initial VCD (10^6 cell/mL): VCD at time t = 0

## Generate Experiment Run

In [102]:
# Model parameters
mu_g_max = 0.05;
mu_d_max = 0.025;
K_g_Glc  = 1;
K_I_Lac  = 30;
K_d_Lac  = 50;
k_Glc    = 0.04;
k_Lac    = 0.06;
k_Prod   = 1;
model_param = [mu_g_max,mu_d_max,K_g_Glc,K_I_Lac,K_d_Lac,k_Glc,k_Lac,k_Prod]

# Process parameters
feed_start = 3.0
feed_end = 12.0
Glc_feed_rate = 12.0
Glc_0 = 10.0
VCD_0 = 0.3
process_param = [feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0]


In [103]:
# Experiment run generation
t,y = emulator.predict_chrom_phase(model_param,process_param)
run_df = pd.DataFrame(y,columns=["X:VCD", "X:Glc", "X:Lac", "X:Titer"])
run_df


,X:VCD,X:Glc,X:Lac,X:Titer
0,0.300000,10.000000,0.000000,0.000000
1,0.877666,9.484266,0.777573,0.129431
2,2.389344,8.030174,2.971193,0.931287
3,5.451007,4.419283,8.432332,7.038657
4,9.954081,9.171630,19.380135,32.672120
5,14.236051,9.490923,36.991679,107.606442
6,16.097601,6.789103,59.179200,249.179059
7,15.297378,3.687971,82.056450,427.837006
8,12.870307,2.335591,102.445937,595.285539
9,10.245350,3.474163,119.046093,738.830470


In [104]:
fig = make_subplots(rows=1, cols=4, subplot_titles=list(run_df.columns)[0:4])
for i,c in enumerate(list(run_df.columns)[0:4]):
    fig.add_trace(go.Scatter(x=np.array(t)/24, y=run_df[c].values), row=1, col=i+1)
fig.update_layout(showlegend=False, title_text="X Variable evolution for generated run")
fig.show()

## Generate Design of Experiments

In order to show the complexity of the model, in spite of the apparent simplicity and small number of components, in this section the user can similate the behavior of the process in a broad range of the process variables.

A number of simulations defined by "num_runs" will be generated. For this number of simulations, a latin hypercube design (LHD) is created, to uniformly map the 5-dimensional space of the variables.

### Manipulated Variables

For each of the manipulated variables defined in the section above, the use can define the limits of the exploration space (first value: lower limit; second value: upper limit).

*Note: num_runs is defining the number of simulations. The generated simulations are saved locally in the file "ExperimentsDOE_test.csv", which will be used as test set for the machine learnign tools in the followng lectures.*




In [105]:
''' DOE Dataset definition (variable = [lower bound, upper bound]) '''
# Model parameters
mu_g_max = 0.05;
mu_d_max = 0.025;
K_g_Glc  = 1;
K_I_Lac  = 30;
K_d_Lac  = 50;
k_Glc    = 0.04;
k_Lac    = 0.06;
k_Prod   = 1;

# Process parameters
feed_start = [1, 4]
feed_end = [8, 12]
Glc_feed_rate = [5, 20]
Glc_0 = [10, 80.0]
VCD_0 = [0.1, 1.0]

var_lims = {"mu_g_max":mu_g_max,
    "mu_d_max": mu_d_max,
    "K_g_Glc" : K_g_Glc,
    "K_I_Lac" : K_I_Lac,
    "K_d_Lac" : K_d_Lac,
    "k_Glc" : k_Glc,
    "k_Lac" : k_Lac,
    "k_Prod" : k_Prod,
    "feed_start" : feed_start,
    "feed_end" : feed_end,
    "Glc_feed_rate" : Glc_feed_rate,
    "Glc_0" : Glc_0,
    "VCD_0" : VCD_0}

In [106]:
## Define the number of experiments
num_runs = 50
## Define the filename for the dataset
filename = "OWUExperiments-test.csv"
# Generate DOE Dataset
owu = emulator.generate_data(var_lims, num_runs, filename)

In [107]:
# Read DOE
doe = pd.read_csv("/content/"+filename.replace(".csv","_doe_scaled.csv"),index_col=None, usecols =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"])
doe

,feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0
0,2.500000,10.000000,12.500000,45.000000,0.550000
1,1.948980,10.244898,18.622449,40.714286,0.274490
2,3.295918,9.346939,14.642857,17.857143,0.127551
3,3.479592,9.020408,15.561224,47.857143,0.843878
4,1.459184,9.591837,7.295918,33.571429,0.421429
5,3.418367,9.265306,8.214286,45.000000,0.403061
6,1.153061,8.448980,17.091837,67.857143,0.862245
7,2.438776,8.122449,14.336735,10.714286,0.898980
8,2.744898,11.551020,9.132653,27.857143,0.292857
9,2.316327,9.183673,5.459184,75.000000,0.256122


In [108]:
# Read Dataset
owu = pd.read_csv("/content/"+filename,index_col=0)
owu.columns =  ["X:VCD", "X:Glc", "X:Lac", "X:Titer"]
owu.index = pd.MultiIndex.from_product([list(range(num_runs)),list(range(15))], names=["run","time"])
owu

X:VCD         X:Glc       X:Lac      X:Titer
run time                                                  
0   0      0.550000  4.500000e+01    0.000000     0.000000
    1      1.725244  4.400819e+01    1.489389     0.054632
    2      4.779558  4.108962e+01    5.872389     1.311539
    3     10.278397  4.027148e+01   16.487303    15.870869
    4     15.886799  4.008360e+01   35.542541    85.824621
    5     18.568089  3.578307e+01   60.776450   245.221131
    6     18.099141  3.050520e+01   87.483607   468.616199
    7     15.811653  2.666724e+01  112.033694   706.666289
    8     12.920362  2.538948e+01  132.740373   924.761704
    9     10.123559  2.687354e+01  149.296217  1108.215155
    10     7.715657  3.086152e+01  162.086471  1254.876222
    11     5.775454  2.443517e+01  171.743604  1367.869866
    12     4.271029  1.965583e+01  178.928939  1452.631393
    13     3.129193  1.613956e+01  184.218131  1515.192811
    14     2.276241  1.357394e+01  188.079549  1560.822799
1   0      0.274490  4.071429e+01    0.000000     0.000000
    1      0.871876  4.021714e+01    0.746634     0.016378
    2      2.589262  3.964111e+01    3.038611     0.281785
    3      6.529319  5.409861e+01    9.292550     3.954588
    4     12.160305  6.377184e+01   22.727610    32.231163
    5     15.860577  6.865867e+01   43.346514   126.914091
    6     15.657798  7.187437e+01   66.473083   287.510524
    7     12.758719  7.672515e+01   87.144554   464.795721
    8      8.910175  8.493767e+01  102.769178   614.381177
    9      5.459967  9.673826e+01  113.007662   718.310353
    10     3.082856  1.113521e+02  119.023440   781.305931
    11     1.685314  1.136906e+02  122.359351   816.813063
    12     0.913984  1.124804e+02  124.175404   836.302382
    13     0.493770  1.118254e+02  125.158349   846.896767
    14     0.266218  1.114719e+02  125.688833   852.627575
2   0      0.127551  1.785714e+01    0.000000     0.000000
    1      0.394527  1.763047e+01    0.340964     0.018588
    2      1.182078  1.694057e+01    1.378818     0.127191
    3      3.245514  1.496382e+01    4.353247     1.074543
    4      7.422644  2.035060e+01   11.759685     8.774240
    5     12.957409  2.523241e+01   26.432561    48.749962
    6     16.940040  2.532607e+01   48.298985   162.331614
    7     17.855081  2.306502e+01   73.707149   351.366208
    8     16.416292  2.115971e+01   98.585835   575.721018
    9     13.883034  2.124905e+01  120.468087   795.410123
    10    11.136455  1.435542e+01  138.476060   987.641102
    11     8.556549  4.974991e+00  152.626226  1139.403952
    12     5.898992 -8.613934e-09  163.107556  1203.964705
    13     3.717065 -1.623925e-09  169.913054  1203.964705
    14     2.334220 -7.634924e-09  174.194052  1203.964705
3   0      0.843878  4.785714e+01    0.000000     0.000000
    1      2.609587  4.634350e+01    2.272871     0.131730
    2      6.769304  4.207177e+01    8.687722     3.385544
    3     12.917048  3.268206e+01   22.791228    33.184158
    4     17.673164  2.587774e+01   45.185904   140.090308
    5     18.962614  2.361476e+01   71.976080   334.205459
    6     17.510225  2.155014e+01   98.473705   571.894931
    7     14.783900  2.159490e+01  121.802901   806.697565
    8     11.821098  2.442164e+01  140.946719  1013.069941
    9      9.128513  2.997754e+01  155.982735  1182.667689
    10     6.889822  2.265957e+01  167.457298  1315.528138
    11     5.119027  1.694532e+01  176.050461  1415.968910
    12     3.756825  1.272902e+01  182.396371  1490.197449
    13     2.729721  9.653159e+00  187.030910  1544.060720
    14     1.966637  7.430490e+00  190.384540  1582.546615
4   0      0.421429  3.357143e+01    0.000000     0.000000
    1      1.320115  3.281346e+01    1.138668     0.043076
    2      3.754135  3.449978e+01    4.532848     0.773875
    3      8.637166  3.604517e+01   13.170695     9.559933
    4     14.486637  3.219574e+01   29.913031    60.328294
    5     18.036687  2.363680e+01   53.738055   195.664223
  

## Analyse single experiment

Here you can visualize particular run from the generated dataset. You can change the `selected_run_ix` to decide which run will be plotted.



In [109]:
# Select run id to plot
selected_run_ix = 3

In [110]:
fig = make_subplots(rows=1, cols=4, subplot_titles=list(owu.columns)[0:4])
plot_run_ix = owu.index.get_level_values("run") == selected_run_ix
for i,c in enumerate(list(owu.columns)[0:4]):
    fig.add_trace(go.Scatter(x=list(range(15)), y=owu[c].values[plot_run_ix]), row=1, col=i+1)
fig.update_layout(showlegend=False, title_text="X Variable evolution for run "+str(selected_run_ix))
fig.show()

## Analyse all experiments

Here you can visualize all run from the generated dataset. By changing the `color_run` you decide on the coloring of the experiments. The options are:
* `Run_id` runs are colored by the order in which they appear in the dataset.
* `Titer_14` runs are colored by the amount of Titer at day 14 or the experiments.
* `Glc_0` run are colored by the designed initial Glucose level
* `VCD_0` run are colored by the designed initial VCD level
* `feed_start` run are colored by the designed feeding start day 
* `feed_end` run are colored by the designed feeding end day
* `Glc_feed_rate` run are colored by the designed Glucose feed rate

In [111]:
# Select coloring option (Run_id, Titer_14, Glc_0, VCD_0, feed_start, feed_end, Glc_feed_rate)
color_run = "Run_id"

In [113]:
color_show=True
if color_run == "Titer_14":
    color_idx =np.repeat(np.array(owu["X:Titer"][:,14]),15)
elif color_run == "Glc_0":
    color_idx =np.repeat(np.array(owu["X:Glc"][:,0]),15)
elif color_run == "VCD_0":
    color_idx =np.repeat(np.array(owu["X:VCD"][:,0]),15)
elif color_run == "feed_start":
    color_idx =np.repeat(np.array(doe['feed_start']),15)
elif color_run == "feed_end":
    color_idx =np.repeat(np.array(doe['feed_end']),15)
elif color_run == "Glc_feed_rate":  
    color_idx =np.repeat(np.array(doe['Glc_feed_rate']),15)
else:
    color_run ="Run_id"
    color_idx=np.repeat(np.array(list(range(doe.shape[0]))),15)     
    color_show=False
owu["color"] = color_idx

for i in list(owu.columns)[0:4]:
    fig = go.Figure()
    for color_val in np.unique(color_idx):
        color_val_norm = (color_val -min(color_idx)) / (max(color_idx)-min(color_idx))
        owu_subset = owu[owu['color']==color_val]
        fig.add_trace(go.Scatter(x=owu_subset.index.get_level_values(1),y=owu_subset[i],mode='lines+markers',marker=dict(color=plothelpers.get_color('Plasma',color_val_norm))))
    fig.add_trace(go.Scatter(x=owu.index.get_level_values(1),y=owu[i],mode='markers',marker=dict(size=0,color="rgba(0,0,0,0)",colorscale='Plasma',cmin=min(color_idx),cmax=max(color_idx),colorbar=dict(thickness=40, title=str(color_run))),showlegend=False))
    fig.update_layout(showlegend=False,xaxis_title='days',yaxis_title=i, title=i)
    fig.show()

## Tasks

1. Identify which process parameters produce the highest titer in the simulated experiments